In [59]:
import cv2
import torch 
import os
from skimage.metrics import structural_similarity
from PIL import Image
import PIL
import pandas as pd

In [54]:
cwd = os.getcwd()

In [55]:
get_stem = lambda x: {x.stem}

In [47]:
h5_path = cwd+'/data/h5/'
gt_path = h5_path+'/gt/'
hr_path = h5_path+'/hr/'
naive_path = h5_path+'/naive/'
lanczos_path = h5_path+'/lanczos/'
valid_path = h5_path+'/s2-lr-valid/'
path_lr = h5_path+'/s2/'

In [48]:
valid_list = [f for f in os.listdir(valid_path) if os.path.isfile(os.path.join(valid_path, f))]

In [51]:
for line in valid_list:
    gt = Image.open(path_lr+line)
    lz = gt.resize((400, 400), resample=PIL.Image.LANCZOS)
    lz.save(lanczos_path+line)
    nv = gt.resize((400, 400))
    nv.save(naive_path+line)

In [57]:
results = []
for image in valid_list:
    gt_im = cv2.imread(gt_path+image)
    test1 = Image.open(gt_path+image)
    naive_im = cv2.imread(naive_path+image)
#     try:
#         test2 = Image.open(naive_path+image)
#     except:
#         results.append([image, "naive"])
    lanczos_im = cv2.imread(lanczos_path+image)
#     try:
#         test3 = Image.open(lanczos_path+image)
#     except:
#         results.append([image, "lanczos"])
    hr_im = cv2.imread(hr_path+image)
#     try:
#         test4 = Image.open(hr_path+image)
#     except:
#         results.append([image, "hr"])
    
    
    psnr_naive = cv2.PSNR(gt_im, naive_im)
    psnr_lanczos = cv2.PSNR(gt_im, lanczos_im)
    psnr_hr = cv2.PSNR(gt_im, hr_im)
    
    ssim_naive =  structural_similarity(gt_im, naive_im, multichannel=True)
    ssim_lanczos =  structural_similarity(gt_im, lanczos_im, multichannel=True)
    ssim_hr =  structural_similarity(gt_im, hr_im, multichannel=True)
    
    results.append([image, psnr_hr, psnr_lanczos, psnr_naive, ssim_hr, ssim_lanczos, ssim_naive])

In [58]:
results

[['8286800_5496400.png',
  17.65011176597462,
  19.227093896576683,
  19.311595184173946,
  0.13389607587812904,
  0.21865263642455848,
  0.22193574136781105],
 ['8286800_5496800.png',
  17.9105996902573,
  20.204823983792302,
  20.291253386629705,
  0.09949554886169437,
  0.20746770804482015,
  0.21103246257959296],
 ['8287200_5496400.png',
  18.544080389982167,
  19.52233805043156,
  19.591947441708303,
  0.1829200979846327,
  0.2681259657381206,
  0.27088549147258484],
 ['8287200_5498000.png',
  18.08897787312337,
  19.410478250213686,
  19.479910073748396,
  0.12723892429319847,
  0.2239464098393695,
  0.2256597614933994],
 ['8287600_5497200.png',
  18.16679594081012,
  20.278325304142054,
  20.355299168825333,
  0.14692427828297064,
  0.24237084083787017,
  0.24530985087368537],
 ['8287600_5497600.png',
  17.62163761388593,
  20.011471483793066,
  20.091106925026914,
  0.15309655871610903,
  0.25606089494986145,
  0.2581003962184458],
 ['8288000_5498000.png',
  17.823681436793567,

In [61]:
df = pd.DataFrame(results, columns = ['Loc ID', 'HR PSNR', 'Lanczos PSNR', 'Bicubic PSNR', 'HR SSIM', 'Lanczos SSIM', 'Bicubic SSIM'])

In [62]:
df

,Loc ID,HR PSNR,Lanczos PSNR,Bicubic PSNR,HR SSIM,Lanczos SSIM,Bicubic SSIM
0,8286800_5496400.png,17.650112,19.227094,19.311595,0.133896,0.218653,0.221936
1,8286800_5496800.png,17.910600,20.204824,20.291253,0.099496,0.207468,0.211032
2,8287200_5496400.png,18.544080,19.522338,19.591947,0.182920,0.268126,0.270885
3,8287200_5498000.png,18.088978,19.410478,19.479910,0.127239,0.223946,0.225660
4,8287600_5497200.png,18.166796,20.278325,20.355299,0.146924,0.242371,0.245310
...,...,...,...,...,...,...,...
292,9601600_6069200.png,13.114939,14.932642,15.011943,0.113504,0.162467,0.157931
293,9604000_6066800.png,17.326800,16.737049,16.773876,0.247526,0.286222,0.284810
294,9604000_6076000.png,14.035697,5.050810,5.050821,0.117916,0.133834,0.132592
295,9606000_6070400.png,13.696942,15.690324,15.732821,0.170386,0.191710,0.185552


In [64]:
df.mean()

HR PSNR         17.321502
Lanczos PSNR    16.227159
Bicubic PSNR    16.271802
HR SSIM          0.239568
Lanczos SSIM     0.277140
Bicubic SSIM     0.277273
dtype: float64

In [65]:
df.to_csv('pnsr_ssim.csv')